<a href="https://colab.research.google.com/github/Jinukki/KJU/blob/master/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# seq2seq란?
##### 지금 이 공학전산응용을 듣는 학생들 중 거의 대부분은 조사를 할 때 혹은 과제를 할 때 구글이나 네이버의 번역기를 이용해 본 경험이 있을 것이다. 그 번역이 완전히 매끄러웠는지는 모르겠지만, 어쨌든 그럴싸한 순서를 가진 문장이 번역되어 나온다. 이런 문장들은 어떻게 번역되어 나온 것일까?

##### 자연어 처리 분야가 당면한 과제에서는 다양한 길이의 output 시퀀스가 중요할 때가 많다. 예를 들어 번역, 대화, 혹은 우리가 직접 글을 요약해야할 때 등이 있다. 위 예시들은 단어 한 개만 나오면 끝이 아니라 "문장" 즉, 단어들의 시퀀스가 나와줘야 평가가 가능하다.

##### 이를 위해 seq2seq라 불리는 모델들을 살펴보자. 이러한 모델들은 2014년에 처음 사용되어 나온지 5년 정도 밖에 되지 않았지만 기계 번역 분야에 있어 표준이 되어가고 있다.

##### seq2seq의 장점은 이 모델이 사용하는 LSTM(Long-Short Term Memory)과의 연관성에서 나오는데, 과거의 번역 시스템이 장기 의존성(Long-Term Dependency)를 해결하기 구조였던 것에 비해, seq2seq는 전체 input을 훑어본 뒤에 임의의 어떤 시퀀스를 출력할 수 있으며, 효율적인 번역을 위해 input의 특정 부분만을 살펴볼 수도 있다.

---

##### 여기서 장기 의존성 문제에 설명을 하자면, RNN은 "The clouds are in the sky 문장에서 'sky'를 예측하는 경우" 와 같이 시간적으로 상관관계가 있는 데이터를 사용한다. 아래 그림과 같이 직진 데이터(t-1)과 현재 데이터(t) 간 상관관계를 고려하여 다음 데이터(t+1)를 예측하고자, 과거 데이터도 반영한 신경망 모델을 만든다.

![RNN구조](https://user-images.githubusercontent.com/52277776/69826516-cdf38f00-1256-11ea-923a-eb58b54d5d9c.PNG)

##### 그러나 RNN 처럼 직전 정보만 참고하는 것이 아니라,  그 전 정보를 고려해야 하는 경우(longer-term) 가 있다. <br>
 * 책을 읽을 때, 몇 페이지/챕터 전에 있는 정보를 머리 속에 기억하고 있어야 하는 경우<br>
 * I grew up in France... I speak fluent French. 문장에서 'French'를 예측하는 경우
##### 위의 예에서 마지막에 나온 French는 언어의 이름일 것이다. 그런데 French라는 단어의 의미가 많을텐데, French가 언어의 이름이라는 것을 어떻게 알 수 있을까? 바로 전에 나온 France라는 말에서 그 힌트를 찾을 수 있다. 두 예에서와 같이 전체적으로 앞에서 나온 정보를 고려해야 하는 경우, 문장 간의 간격(gap, 입력 위치의 차이)이 커질 수록, RNN은 두 두 정보의 맥락을 파악하기 어려워지는데 이를 해결하기 위해서 고안된 것이 LSTM이다. LSTM은 Tensorflow에서 구현 가능하다.

---

##### LSTM에 대한 설명은 발표에서 진행하였으므로 생략하고 seq2seq 이야기로 넘어가도록 하자. 앞에서 언급하였듯이 2014년 당시 불어-영어 번역을 위해 처음 사용 되었으며 새로운 패러다임의 등장이었다. 2개의 RNN모델을 합쳐 놓은 것으로서, 아래 그림과 같이 encoder, decoder로 이루어져 있다.

![seq2seq base 구조](https://user-images.githubusercontent.com/52277776/69826512-c92edb00-1256-11ea-91db-e8efb3123d99.PNG)

 * encoder에서는 기본적으로 모델의 input 시퀀스로부터 고정된 사이즈의 context vector
를 만든다.<br>
 * decoder에서는 이 context vector를 다시 input으로 하여 번역된 output 시퀀스를 만든다.

##### 이러한 구조 때문에 Seq2Seq는 종종 “encoder decoder model”이라고 불린다.

<br>

 > #### encoder에 대해 알아보자. 
 
![s2s encoder](https://user-images.githubusercontent.com/52277776/69826523-d4820680-1256-11ea-8d3d-e349d933bbff.PNG)
 
#### 인코더의 역할은 그림에서 보다시피 input 시퀀스를 하나의 context vector C로 만드는 일이다. 이렇게 input 단어를 한 개씩 읽기 위해서 인코더는 LSTM을 주로 활용한다. input 시퀀스의 길이가 그 때 그 때 바뀌지만 항상 일정한 길이의 C로 시퀀스를 압축해야 한다는 어려움이 있기 때문에 인코더는 대부분 그림처럼 stacked LSTM으로 구성한다. 이 LSTM layer들은 이전 layer의 output을 다음 layer로 전달하면서 input 시퀀스를 성공적으로 압축하는데 사용된다. 참고로 그림에서 자세히 나와있지만, “name your is what” 순서로 시퀀스를 뒤집어서 입력하게 되는데, 이는 첫 단어를 마지막으로 입력함으로서, 후에 decoder가 첫 단어인 what 부터 잘 번역할 수 있도록 하려는 의도이다. decoder는 일단 what이 잘 번역되면 뒷 단어들도 이의 영향으로 차례로 잘 번역될 수 있도록 설계되어 있다.
<br>

 > #### decoder에 대해 알아보자. 

![s2s decoder](https://user-images.githubusercontent.com/52277776/69826524-d77cf700-1256-11ea-8cfb-95cf9082e22a.PNG)

 ##### 디코더의 구조를 살펴보면 인코더와 마찬가지로 stacked LSTM구조를 사용하는 것은 동일하다. 하지만 아래와 같은 두 가지 큰 차이가 있다.
 * 이 네트워크를 인코딩 완료된 context vector로 초기화 한다는 점<br>
 * stacked LSTM의 마지막 레이어로부터 나온 output word를 다시 다음 input에 사용한다는 점<br>

##### 그렇다면 input과 output에 활용되는 시퀀스의 길이가 각각 다를텐데, 어떻게 이렇게 다른 길이의 output이 가능할까? 이 것은 아래 그림을 보면 더 쉽게 이해할 수 있는데, 인코더와 디코더에서 라는 토큰을 사용하여 문장의 입력과 출력이 끝남을 네트워크에 전달한다.

![s2s 종합구조](https://user-images.githubusercontent.com/52277776/69826528-da77e780-1256-11ea-8373-23027af11129.PNG)

##### 여기서 또 주목 할만한 점은 인코더와 디코더가 동시에 학습된다는 점이다. 위와 같이 옆으로 긴 인코더, 디코더가 한꺼번에 forward-propagate(순전파)되었다가 loss function(손실함수)에 근거한 back-propagation(역전파)을 통해 이 네트워크에 사용되는 모든 파라미터가 1번 업데이트 된다.

##### 그런데 시퀀스의 입력과 출력 길이에 어떠한 제약도 없다는 것은 사실이지만, Seq2Seq 모델은 LSTM의 한계 때문에 입력 문장이 매우 길면 효율적으로 학습하지 못한다는 사실이 알려져 있다.

###### 문장 안의 의존 관계는 그저 한 쪽 방향으로 학습을 진행해서는 제대로 학습이 되지 않는다. 현재 위에서 설명한 Seq2Seq 네트워크는 양 방향 RNN을 반영하지 못하고 한 쪽 방향으로만 진행되고 있는 형태이다. 인코더를 어떻게 개선하여 Seq2Seq를 개선할 수 있을까?

##### Bidirectional RNN은 입력 시퀀스가 양방향으로 입력될 수 있게 아래와 같은 구조를 고안했다.

![bidirectional RNN](https://user-images.githubusercontent.com/52277776/69826529-dcda4180-1256-11ea-843d-35be14f3cf3b.PNG)

##### 여기서 최종 hidden state은 정방향, 역방향으로 학습된 아래 식과 같은 두 개의 벡터를 이어 붙인 형태로 계산된다.
$$h=[h^{(forward)};h^{(backward)}]$$